In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 11 22:52:51 2017

@author: Tu Bui tb00083@surrey.ac.uk
"""

import sys,os
from PIL import Image
import StringIO
import math
import subprocess
import caffe
import numpy as np
from sklearn.metrics import confusion_matrix
from scipy.io import savemat
import time
from os import listdir
from os.path import isfile, join
import os

MODEL_WEIGHTS_PATH = 'model/triplet1_InceptionV1_InceptionV1_halfshare_inception4e_ld256_triplet_sketchy_iter_31200.caffemodel'
MODEL_SPEC_PATH = 'model/deploy_images_net1_InceptionV1_InceptionV1_halfshare_inception4e_ld256_triplet_sketchy.prototxt'



GPU_DEV = 0
LAYER_DIMS=256
mean_pixel = np.array([104, 117, 123],dtype=np.float32)[:,None,None]

def get_net(caffemodel, deploy_file, use_gpu=True):
    """
    Returns an instance of caffe.Net

    Arguments:
    caffemodel -- path to a .caffemodel file
    deploy_file -- path to a .prototxt file

    Keyword arguments:
    use_gpu -- if True, use the GPU for inference
    """
    if use_gpu:
        caffe.set_mode_gpu()
        caffe.set_device(GPU_DEV)

    # load a new model
    return caffe.Net(deploy_file, caffe.TEST, weights = caffemodel)


def extractitem(net, mean_pixel, fname):
  
    DATA_LAYER = net.inputs[0]
    net.blobs[DATA_LAYER].reshape(1,3,224,224) 
    try:
       input_image = Image.fromarray(np.uint8(caffe.io.load_image(fname)*255))#.resize((256,256),Image.BILINEAR).crop((16,16,240,240))
       #resize
       sf = 256.0/max(input_image.size)
       input_image = input_image.resize((int(input_image.width*sf),int(input_image.height*sf)),Image.BILINEAR)
       pw = (256 - input_image.width)/2
       ph = (256 - input_image.height)/2
       input_image = np.array(input_image)
         
       #make sure image is RGB format
       if input_image.ndim == 2:
         input_image = input_image[...,None]
         input_image = np.repeat(input_image,3, axis=2)
          
       #pad to make 256x256
       input_image = np.pad(input_image,((ph,ph),(pw,pw),(0,0)), mode='edge')
         
       #crop
       input_image = input_image[16:240,16:240]
       
       sys.stdout.flush()
       transformed_image = np.array(input_image,dtype=np.float32)[:,:,::-1].transpose(2,0,1) - mean_pixel
       sys.stdout.flush()
       net.blobs[DATA_LAYER].data[...] = transformed_image
       sys.stdout.flush()
       _ = net.forward()
       sys.stdout.flush()
       blobname=net.blobs.keys()[-1] #should be feat_p for image and feat_a for sketch
       prediction = net.blobs[blobname].data.squeeze()
    
    
    except Exception as e:
       s=str(e)
       print('WARNING: Image was unusable %s' % fname)
       print(s)
       prediction = np.zeros(LAYER_DIMS).astype(np.float32)
    
    return prediction

onlyfiles = [f for f in listdir('test_images') if isfile(join('test_images', f)) and f.endswith(".jpg")]

for file in onlyfiles:
    if __name__ == "__main__":
        net = get_net(MODEL_WEIGHTS_PATH, MODEL_SPEC_PATH)
        sample_img = 'test_images/{}'.format(file)
        feat = extractitem(net, mean_pixel, sample_img)
        np.save(os.path.join('test_images','{}_np'.format(file)), feat)
        
        
        

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:49: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')


In [15]:
pwd

u'/root/SBIR_regression'

In [3]:
import sys,os
from PIL import Image
import StringIO
import math
import subprocess
import caffe
import numpy as np
from sklearn.metrics import confusion_matrix
from scipy.io import savemat
import time
from os import listdir
from os.path import isfile, join
import os

In [4]:
onlyfiles = [f for f in listdir('test_images') if isfile(join('test_images', f))]


In [5]:
net = get_net(MODEL_WEIGHTS_PATH, MODEL_SPEC_PATH)
sample_img = 'il_340x270.2410648087_in8x.jpg'
feat = extractitem(net, mean_pixel, sample_img)
np.save(os.path.join('test_images','{}_np'.format(file)), file)

['il_340x270.2410648087_in8x.jpg',
 'il_340x270.2073936464_c0j0.jpg',
 'il_340x270.1907914799_rvna.jpg',
 'il_340x270.2364392820_jhhv.jpg',
 'il_340x270.2353030942_qd7q.jpg',
 'il_340x270.2342974336_81tc.jpg',
 'il_340x270.2372003330_39vd.jpg',
 'il_340x270.2067739430_8ors.jpg',
 'il_340x270.1278282102_ckx9.jpg',
 'il_340x270.1469950864_a14i.jpg',
 'il_340x270.2287712534_dwqp.jpg',
 'il_340x270.2275212971_nqf3.jpg',
 'il_340x270.2131647248_1ya8.jpg',
 'il_340x270.2371102015_1n9f.jpg',
 'il_340x270.2391984655_ctge.jpg',
 'il_340x270.2160916622_bzm1.jpg',
 'il_340x270.1507866627_xbeg.jpg',
 'il_340x270.2309314045_gbhs (1).jpg',
 'il_340x270.512342129_cwho.jpg',
 'il_340x270.2381971296_ahgy.jpg',
 'il_340x270.2387321984_q3ji.jpg',
 'il_340x270.1324558183_ilrw.jpg',
 'il_340x270.1496900707_pi5k.jpg',
 'il_340x270.2199940746_f651.jpg',
 'il_340x270.747124501_6ye7.jpg',
 'il_340x270.1999471569_su61.jpg',
 'il_340x270.2278053371_dbml.jpg',
 'il_340x270.1325624325_3qgn.jpg',
 'il_340x270.24095

In [17]:
arr=np.load('test_images/il_340x270.955601918_rvfm.jpg_np.npy')

In [20]:
arr

array([-0.03651689, -0.0612675 ,  0.01531431, -0.01320246,  0.00496379,
        0.0399141 , -0.00268625,  0.03471153,  0.02592528,  0.10695248,
        0.04638259, -0.04752885,  0.06952831, -0.02184477,  0.13420334,
        0.0526466 ,  0.03620138,  0.02073135, -0.09035253, -0.07611809,
       -0.01881354, -0.03986524, -0.04154817,  0.07388154,  0.00038265,
       -0.04911083,  0.01595832,  0.02672974, -0.12276614,  0.04299167,
       -0.00030625, -0.07371964,  0.10821757, -0.05371689, -0.0289405 ,
       -0.10275897, -0.06078856,  0.08838732, -0.02804676,  0.03126766,
        0.08846602, -0.09880531, -0.02764767,  0.06170678, -0.03105168,
        0.00350754, -0.05579326,  0.0378186 , -0.00748581, -0.05755332,
       -0.0933305 ,  0.02974426,  0.03267759,  0.03151875,  0.11558694,
       -0.11887546,  0.05341901, -0.03423889,  0.02872844, -0.0785173 ,
       -0.0032832 , -0.0392836 ,  0.08368971,  0.05034567, -0.10492387,
        0.10651879,  0.01609247, -0.06875584,  0.04373863,  0.02